In [2]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [6]:
with open('/Users/ankushbharadwaj/Desktop/News_dataset.pickle','rb') as data:
    df = pickle.load(data)

In [7]:
df.head()

,Category,File_Name,Content,Complete_Filname,id,News_length
0,business,001.txt,Ad sales boost Time Warner profit\n\nQuarterly...,001.txt-business,1,2558
1,business,002.txt,Dollar gains on Greenspan speech\n\nThe dollar...,002.txt-business,1,2251
2,business,003.txt,Yukos unit buyer faces loan claim\n\nThe owner...,003.txt-business,1,1550
3,business,004.txt,High fuel prices hit BA's profits\n\nBritish A...,004.txt-business,1,2400
4,business,005.txt,Pernod takeover talk lifts Domecq\n\nShares in...,005.txt-business,1,1568


In [10]:
df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content'].str.replace("   ", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('""','')

In [11]:
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

In [19]:
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']
for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s","")

In [20]:
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ankushbharadwaj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ankushbharadwaj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
wordnet_lem = WordNetLemmatizer()

In [23]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    lemmatized_list = []
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")
    for word in text_words:
        lemmatized_list.append(wordnet_lem.lemmatize(word, pos="v"))
    lemmatized_text = " ".join(lemmatized_list)
    lemmatized_text_list.append(lemmatized_text)

In [24]:
df['Content_Parsed_5'] = lemmatized_text_list

In [25]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ankushbharadwaj/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [26]:
stop_words = list(stopwords.words('english'))
stop_words[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [27]:
df['Content_Parsed_6'] = df['Content_Parsed_5']
for stop_word in stop_words:
    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword,"")

In [28]:
df.head()

,Category,File_Name,Content,Complete_Filname,id,News_length,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,business,001.txt,Ad sales boost Time Warner profit\n\nQuarterly...,001.txt-business,1,2558,Ad sales boost Time Warner profit\n\nQuarterly...,ad sales boost time warner profit\n\nquarterly...,ad sales boost time warner profit\n\nquarterly...,ad sales boost time warner profit\n\nquarterly...,ad sales boost time warner profit\n\nquarterly...,ad sales boost time warner profit\n\nquarterly...
1,business,002.txt,Dollar gains on Greenspan speech\n\nThe dollar...,002.txt-business,1,2251,Dollar gains on Greenspan speech\n\nThe dollar...,dollar gains on greenspan speech\n\nthe dollar...,dollar gains on greenspan speech\n\nthe dollar...,dollar gains on greenspan speech\n\nthe dollar...,dollar gain on greenspan speech\n\nthe dollar ...,dollar gain greenspan speech\n\n dollar hit ...
2,business,003.txt,Yukos unit buyer faces loan claim\n\nThe owner...,003.txt-business,1,1550,Yukos unit buyer faces loan claim\n\nThe owner...,yukos unit buyer faces loan claim\n\nthe owner...,yukos unit buyer faces loan claim\n\nthe owner...,yukos unit buyer faces loan claim\n\nthe owner...,yukos unit buyer face loan claim\n\nthe owners...,yukos unit buyer face loan claim\n\n owners e...
3,business,004.txt,High fuel prices hit BA's profits\n\nBritish A...,004.txt-business,1,2400,High fuel prices hit BA's profits\n\nBritish A...,high fuel prices hit ba's profits\n\nbritish a...,high fuel prices hit ba's profits\n\nbritish a...,high fuel prices hit ba profits\n\nbritish air...,high fuel price hit ba profits\n\nbritish airw...,high fuel price hit ba profits\n\nbritish airw...
4,business,005.txt,Pernod takeover talk lifts Domecq\n\nShares in...,005.txt-business,1,1568,Pernod takeover talk lifts Domecq\n\nShares in...,pernod takeover talk lifts domecq\n\nshares in...,pernod takeover talk lifts domecq\n\nshares in...,pernod takeover talk lifts domecq\n\nshares in...,pernod takeover talk lift domecq\n\nshares in ...,pernod takeover talk lift domecq\n\nshares uk...


In [32]:
list_of_final_columns = ["File_Name", "Category", "Complete_Filname", "id", "News_length", "Content", "Content_Parsed_6"]

In [37]:
df = df[list_of_final_columns]
df = df.rename(columns={'Content_Parsed_6':'Content_Parsed', "Complete_Filname":"Complete_Filename"})

In [38]:
df.head()

,File_Name,Category,Complete_Filename,id,News_length,Content,Content_Parsed
0,001.txt,business,001.txt-business,1,2558,Ad sales boost Time Warner profit\n\nQuarterly...,ad sales boost time warner profit\n\nquarterly...
1,002.txt,business,002.txt-business,1,2251,Dollar gains on Greenspan speech\n\nThe dollar...,dollar gain greenspan speech\n\n dollar hit ...
2,003.txt,business,003.txt-business,1,1550,Yukos unit buyer faces loan claim\n\nThe owner...,yukos unit buyer face loan claim\n\n owners e...
3,004.txt,business,004.txt-business,1,2400,High fuel prices hit BA's profits\n\nBritish A...,high fuel price hit ba profits\n\nbritish airw...
4,005.txt,business,005.txt-business,1,1568,Pernod takeover talk lifts Domecq\n\nShares in...,pernod takeover talk lift domecq\n\nshares uk...


In [39]:
category_labels = {
    'business':0,
    'entertainment':1,
    'politics':2,
    'sport':3,
    'tech':4
}

In [41]:
df['Category_Code'] = df['Category']
df = df.replace({'Category_Code':category_labels})

In [42]:
df.head()

,File_Name,Category,Complete_Filename,id,News_length,Content,Content_Parsed,Category_Code
0,001.txt,business,001.txt-business,1,2558,Ad sales boost Time Warner profit\n\nQuarterly...,ad sales boost time warner profit\n\nquarterly...,0
1,002.txt,business,002.txt-business,1,2251,Dollar gains on Greenspan speech\n\nThe dollar...,dollar gain greenspan speech\n\n dollar hit ...,0
2,003.txt,business,003.txt-business,1,1550,Yukos unit buyer faces loan claim\n\nThe owner...,yukos unit buyer face loan claim\n\n owners e...,0
3,004.txt,business,004.txt-business,1,2400,High fuel prices hit BA's profits\n\nBritish A...,high fuel price hit ba profits\n\nbritish airw...,0
4,005.txt,business,005.txt-business,1,1568,Pernod takeover talk lifts Domecq\n\nShares in...,pernod takeover talk lift domecq\n\nshares uk...,0


In [43]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'],
                                                   df['Category_Code'],
                                                   test_size=.15,
                                                   random_state=8)

In [44]:
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [45]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(1891, 300)
(334, 300)


In [47]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_labels.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'business' category:
  . Most correlated unigrams:
. firm
. economy
. oil
. growth
. bank
  . Most correlated bigrams:
. last year
. year old

# 'entertainment' category:
  . Most correlated unigrams:
. tv
. music
. award
. star
. film
  . Most correlated bigrams:
. mr blair
. prime minister

# 'politics' category:
  . Most correlated unigrams:
. minister
. blair
. party
. election
. labour
  . Most correlated bigrams:
. prime minister
. mr blair

# 'sport' category:
  . Most correlated unigrams:
. win
. side
. game
. team
. match
  . Most correlated bigrams:
. say mr
. year old

# 'tech' category:
  . Most correlated unigrams:
. digital
. technology
. computer
. software
. users
  . Most correlated bigrams:
. year old
. say mr



In [54]:
with open('X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
with open('X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
with open('y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
with open('y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
with open('df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
with open('features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

with open('labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

with open('features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

with open('labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
with open('tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)